# Cooperation in Networks with Prisioner Dilemma

Made by:
* Eugenio Scafati (IST-ULISBOA student 97959)
* Ricardo Martins (IST-ULISBOA student 67869)

## Questions to Answer

* How does the topology of the network promotes cooperation between peers?
* What is the influence of the temptation to defect in the sustainability of cooperation?

## Network Characteristics
Concepts:

Models: 
1. SFN - Barabasi Albert (Z, m, seed)
2. Watts–Strogatz Regular Graph (p=0)
3. Watts–Strogatz Random Graph (p=1)				


Network chacateristics:
* AVG degree suggested =4
* Size of network = 1_000
* Number of generations to pass trasient time: 10_000
* Number of supervised generations: 1_000 

Prisioners' Dilema: (C - Cooperator; D - defector)

T=b>R=1>S=P=0


Free Parameters:
* Topology - SFN (BA), Regular, Random
* b ]1,2[
* Distribution of cooperators and defectors in the BA network




# Import Dependencies

In [ ]:
# Import Dependencies
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
random.seed(246)  
import time
import datetime
import collections
print('Imports sucessful')

## Give access to drive
To allow the results to be printed in the output file

In [ ]:
# Allow access to the drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
filepath="/content/drive/My Drive/Colab Notebooks/NS_proj2_output_files/NS_Proj2_output_file.txt" 
folder_output="/content/drive/My Drive/Colab Notebooks/NS_proj2_output_files/graphs/" 

# Parameters

In [ ]:
# Define the parameters and variables

num_simulations=100   # NEVER CHANGE THIS VALUE IT IS USED TO DEFINE THE SEED OF THE NETWORK
N=10**3
number_edges=2

number_of_generations=10**4         # Analyze the evolution in stationary stage
supervised_generations =10**3

# Prisioners Dilema Payoffs
T=1.7 # 1<T<2                                           UPDATE HERE
R=1
P=0
S=0

# Functions

In [ ]:
# Prisioner Dilema
def pd_game(my_opinion, opponent_opinion):
  if my_opinion== 'C':
    if opponent_opinion =='C':
      return R
    else:
      return S
  else:
    if opponent_opinion =='C':
      return T
    else:
      return P

In [ ]:
def print2file(topology,sim_ref,seed,time_spent,fraction_of_cooperators):
  # OPEN FILE
  file1 = open(filepath,"a") 

  # WRITE stuff
  print(f'{datetime.datetime.now()},',
        f'{topology}, ',
        f'{sim_ref:4d}, ',
        f'{seed:2d}, ',
        f'{time_spent:0.6f}, ',
        f'{N}, ',
        f'{number_of_generations}, ',
        f'{supervised_generations}, ',
        f'{R}, ',
        f'{T}, ',
        f'{P}, ',
        f'{S}, ',
        f'{fraction_of_cooperators:0.4f}',
        file=file1)

  #CLOSE FILE
  file1.close()  

In [ ]:
def print_filename2file(filepath,filename):
  # OPEN FILE
  file1 = open(filepath,"a") 

  # WRITE stuff
  print(f'{datetime.datetime.now()},',
        f'{filename}',
        file=file1)

  #CLOSE FILE
  file1.close()  

# SIMULATION



*   FOR cicle over simultations - number defined at the begining
   1.   CREATES THE BA GRAPH - to be updated   
   2.   FOR over the transient generations
   3.   FOR over the supervised generations saving qt. of cooperators
   4.   Prints the result in the output file.


In [ ]:
tic_master=time.perf_counter()

# Iterate over a couple simulations
for sim in range(num_simulations):
  
  # The total number of simulations is divided into 5 different newtorks
  # for each topology. This is done controlling the seed
  seed=21+sim//(num_simulations//5) # This is the integer division 
# ______________________________________________________________________________
  # --------------------------------------------------- select one
  # Create the Network                          -
  G = nx.barabasi_albert_graph(N, number_edges, seed = seed); name='BA_RAND'
  #G = nx.watts_strogatz_graph(N, k = 4, p = 1, seed = seed); name='NWS_1-model'
  #G = nx.watts_strogatz_graph(N, k = 4, p = 0, seed = seed); name='NWS-model'
  #G = nx.random_regular_graph(n = N, d = 4, seed = seed); name='Regular-model'

# ______________________________________________________________________________

  # Set the timer for control
  tic = time.perf_counter()

  # --------------------------------------------------- DISTRIBUTION STRATEGY
  # --------------------------------------------------- select one (A or B)

  # STRATEGY A
  # Distribute 50% cooperators and 50% Defectors in the network
  states = np.random.choice(['C','D'], size=len(G.nodes()))
  nx.set_node_attributes(G, dict(zip(G.nodes(), states)), 'state')

  # STRATEGY B
#  # Create a copy of the network to analyse different distribution of defectors
#  name='BA__MOD'
#  # Distribute 50% cooperators and 50% Defectors in the network
#  states = np.random.choice(['C','D'], size=len(G.nodes()))
#  nx.set_node_attributes(G, dict(zip(G.nodes(), states)), 'state')
#  #Sort according to degree
#  sorted_degrees = sorted(G.degree, key=lambda x: x[1], reverse=True)
#  #Make the first 50% detractors and the last 50% cooperators
#  for i in range(len(sorted_degrees)//100):
#    G.nodes[sorted_degrees[i][0]]['state'] = 'D' 



  # --------------------------------------------------- TRASIENT STAGE
  # Iterate over the generations:
  for g in range(number_of_generations):
    # Play the PD to evaluate the fitness
    cum_payoff=np.zeros(len(G.nodes()))
    for u,v,c in G.edges(data=True):   # u is the first node; v is the second node
      cum_payoff[u] += pd_game(G.nodes(data=True)[u]['state'],
                               G.nodes(data=True)[v]['state'])
      cum_payoff[v] += pd_game(G.nodes(data=True)[v]['state'],
                               G.nodes(data=True)[u]['state'])
    
    # Replicate the strategy
    for node in G.nodes():
      neigs=list(G[node])
      neig_selected=random.choice(neigs)
    
      payoff_diff=cum_payoff[neig_selected]-cum_payoff[node]

      # Verify the condition for updating the value
      if payoff_diff>0:
        # Estimate the probability p
        k_higher=np.max([len(neigs),len(list(G[neig_selected]))])
        prob=payoff_diff/(k_higher*(T-S))
      
        # Update the strategy with probability p
        if random.random() < prob:
          G.nodes[node]['state']=G.nodes[neig_selected]['state']
        #else: do nothing

  # --------------------------------------------------- SUPERVISED STAGE
  # Initialize the num_cooperators counter
  num_cooperators=np.zeros(supervised_generations)
  # Iterate over the SUPERVISED generations:
  for g in range(supervised_generations):
    # Play the PD to evaluate the fitness
    cum_payoff=np.zeros(len(G.nodes()))
    for u,v,c in G.edges(data=True):   # u is the first node; v is the second node
      cum_payoff[u] += pd_game(G.nodes(data=True)[u]['state'],
                               G.nodes(data=True)[v]['state'])
      cum_payoff[v] += pd_game(G.nodes(data=True)[v]['state'],
                               G.nodes(data=True)[u]['state'])

    # Replicate the strategy
    for node in G.nodes():
      neigs=list(G[node])
      neig_selected=random.choice(neigs)
    
      payoff_diff=cum_payoff[neig_selected]-cum_payoff[node]

      # Verify the condition for updating the value
      if payoff_diff>0:
        # Estimate the probability p
        k_higher=np.max([len(neigs),len(list(G[neig_selected]))])
        prob=payoff_diff/(k_higher*(T-S))
      
        # Update the strategy with probability p
        if random.random() < prob:
          G.nodes[node]['state']=G.nodes[neig_selected]['state']
      #else: do nothing

    # Store the number of cooperators
    for node in G.nodes():
      if G.nodes(data=True)[node]['state']=='C':
        num_cooperators[g] +=1

  # Evaluate the time of this simulation
  toc = time.perf_counter()
  # Print the results in the output file
  print2file(name,sim,seed,toc-tic,np.mean(num_cooperators/len(G.nodes())))
  # Print info regarding the execution time and sim_number
  print(f"Simulation {sim:4d}.RAND.  Time spent: {toc - tic:0.4f} seconds   Fraction of coop.=",
        np.around(np.mean(num_cooperators/len(G.nodes())),4))

# __end sim loop

# Evaluate and print the total time of simulation.
toc_master = time.perf_counter()
print(f"Time spent: {toc_master - tic_master:0.4f} seconds")
print('TASK FINISHED. RESULTS OUTPUT IN FILE:\n', filepath)

Results are printed in the output files